In [ ]:

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.gridspec as grid_spec
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 31.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing

The goal is to reshape the dataset from a wide to long format. At the beggining we are working with 206 rows and 1008 columns, at the end we will have 6592 rows and 38 columns.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Progetto DS Lab/HDR21-22_Composite_indices_complete_time_series.csv')

In [ ]:
df.shape

(206, 1008)

In [ ]:
df.head()

,iso3,country,hdicode,region,hdi_rank_2021,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
0,AFG,Afghanistan,Low,SA,180.0,0.273,0.279,0.287,0.297,0.292,...,1.86,1.88,1.66,1.62,1.66,1.41,1.32,1.38,1.38,1.38
1,AGO,Angola,Medium,SSA,148.0,NaN,NaN,NaN,NaN,NaN,...,4.09,4.53,3.97,3.59,2.79,2.64,2.28,2.18,2.18,2.18
2,ALB,Albania,High,ECA,67.0,0.647,0.629,0.614,0.617,0.624,...,12.44,11.49,13.14,12.61,14.39,14.46,12.85,12.96,12.96,12.96
3,AND,Andorra,Very High,NaN,40.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,United Arab Emirates,Very High,AS,26.0,0.728,0.739,0.742,0.748,0.755,...,49.56,49.68,55.49,59.76,64.95,75.61,65.97,68.95,68.95,68.95


In [ ]:
df1 = df.filter(like='rank')
df2 = pd.DataFrame({'iso3': [x for x in df.iso3],
                    'country': [x for x in df.country],
                    'hdicode': [x for x in df.hdicode],
                    'region': [x for x in df.region]})

# this dataset contains the static information about a country
country_info = pd.concat([df2, df1], axis =1)

In [ ]:
country_info.head()

,iso3,country,hdicode,region,hdi_rank_2021,gii_rank_2021,rankdiff_hdi_phdi_2021
0,AFG,Afghanistan,Low,SA,180.0,167.0,5.0
1,AGO,Angola,Medium,SSA,148.0,136.0,10.0
2,ALB,Albania,High,ECA,67.0,39.0,28.0
3,AND,Andorra,Very High,NaN,40.0,NaN,NaN
4,ARE,United Arab Emirates,Very High,AS,26.0,11.0,-105.0


In [ ]:
df.drop(columns = country_info.columns, inplace=True)

In [ ]:
df.shape

(206, 1001)

In [ ]:
df.head()

,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,hdi_1995,hdi_1996,hdi_1997,hdi_1998,hdi_1999,...,mf_2012,mf_2013,mf_2014,mf_2015,mf_2016,mf_2017,mf_2018,mf_2019,mf_2020,mf_2021
0,0.273,0.279,0.287,0.297,0.292,0.310,0.319,0.323,0.324,0.332,...,1.86,1.88,1.66,1.62,1.66,1.41,1.32,1.38,1.38,1.38
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.364,...,4.09,4.53,3.97,3.59,2.79,2.64,2.28,2.18,2.18,2.18
2,0.647,0.629,0.614,0.617,0.624,0.634,0.645,0.642,0.657,0.669,...,12.44,11.49,13.14,12.61,14.39,14.46,12.85,12.96,12.96,12.96
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.728,0.739,0.742,0.748,0.755,0.762,0.767,0.773,0.779,0.787,...,49.56,49.68,55.49,59.76,64.95,75.61,65.97,68.95,68.95,68.95


In [ ]:
pattern = re.compile(r'\d{4}')
years = []

for col in df.columns:
    match = pattern.findall(col)
    if match:
        years.append(int(match[0]))

In [ ]:
# regex to remove all years from the indicators

pattern = re.compile(r'^(.*?)_\d{4}$')
df.columns = [pattern.match(col).group(1) for col in df.columns]

In [ ]:
df_T = df.T
df_T.columns = [x for x in country_info['country'].values]
df_T.reset_index(inplace=True)
df_T['edition'] = [x for x in years]

In [ ]:
df_T

,index,Afghanistan,Angola,Albania,Andorra,United Arab Emirates,Argentina,Armenia,Antigua and Barbuda,Australia,...,Medium human development,Low human development,Arab States,East Asia and the Pacific,Europe and Central Asia,Latin America and the Caribbean,South Asia,Sub-Saharan Africa,World,edition
0,hdi,0.273,NaN,0.647,NaN,0.728,0.723,0.656,NaN,0.865,...,0.453000,0.356000,0.555000,0.507000,0.664000,0.633000,0.442000,0.407000,0.601000,1990
1,hdi,0.279,NaN,0.629,NaN,0.739,0.730,0.649,NaN,0.867,...,0.456000,0.359000,0.558000,0.514000,0.661000,0.638000,0.447000,0.410000,0.604000,1991
2,hdi,0.287,NaN,0.614,NaN,0.742,0.735,0.618,NaN,0.869,...,0.461000,0.361000,0.565000,0.524000,0.654000,0.643000,0.453000,0.410000,0.607000,1992
3,hdi,0.297,NaN,0.617,NaN,0.748,0.739,0.616,NaN,0.873,...,0.465000,0.363000,0.574000,0.534000,0.652000,0.648000,0.457000,0.411000,0.610000,1993
4,hdi,0.292,NaN,0.624,NaN,0.755,0.744,0.620,NaN,0.873,...,0.470000,0.366000,0.582000,0.542000,0.647000,0.654000,0.462000,0.413000,0.614000,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,mf,1.410,2.64,14.460,NaN,75.610,18.750,7.350,NaN,48.270,...,4.965644,3.133721,11.626753,17.084116,13.618078,13.604555,4.740263,3.256156,12.277878,2017
997,mf,1.320,2.28,12.850,NaN,65.970,16.880,6.010,NaN,47.700,...,5.151853,3.137389,12.114102,16.961132,13.914030,13.151315,5.000024,3.145930,12.220286,2018
998,mf,1.380,2.18,12.960,NaN,68.950,15.810,5.950,NaN,46.820,...,5.265184,3.147451,12.318860,17.438379,14.087293,13.296195,5.110230,3.151076,12.375236,2019
999,mf,1.380,2.18,12.960,NaN,68.950,15.810,5.950,NaN,46.820,...,5.263171,3.146500,12.249970,17.416781,14.091826,13.290914,5.106447,3.146092,12.325166,2020


In [ ]:
df_long = pd.melt(
    df_T,
    id_vars=['index', 'edition'],
    var_name='country',
    value_name='value'
)


df_long = df_long.sort_values(by=['country', 'edition'])

In [ ]:
df_unstacked = df_long.pivot_table(
    index=['country', 'edition'],
    columns='index',
    values='value'
).reset_index()

In [ ]:
df_unstacked.columns

Index(['country', 'edition', 'abr', 'co2_prod', 'coef_ineq', 'diff_hdi_phdi',
       'eys', 'eys_f', 'eys_m', 'gdi', 'gdi_group', 'gii', 'gni_pc_f',
       'gni_pc_m', 'gnipc', 'hdi', 'hdi_f', 'hdi_m', 'ihdi', 'ineq_edu',
       'ineq_inc', 'ineq_le', 'le', 'le_f', 'le_m', 'lfpr_f', 'lfpr_m', 'loss',
       'mf', 'mmr', 'mys', 'mys_f', 'mys_m', 'phdi', 'pr_f', 'pr_m', 'se_f',
       'se_m'],
      dtype='object', name='index')

To check if the values have been correctly transposed:

In [ ]:
country_info.tail()

,iso3,country,hdicode,region,hdi_rank_2021,gii_rank_2021,rankdiff_hdi_phdi_2021
201,ZZG.ECA,Europe and Central Asia,NaN,NaN,NaN,NaN,NaN
202,ZZH.LAC,Latin America and the Caribbean,NaN,NaN,NaN,NaN,NaN
203,ZZI.SA,South Asia,NaN,NaN,NaN,NaN,NaN
204,ZZJ.SSA,Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN
205,ZZK.WORLD,World,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Values of hdi in the orginal dataframe vs new dataframe are the same
df.iloc[0:1,0:32].T.reset_index(drop=True)[0].equals(df_unstacked[df_unstacked['country'] == 'Afghanistan']['hdi'])

True

#Preprocessing null values. ONLY 2021 FOR COVARIATES ANALYSIS

In [ ]:
#Prendo solo i dati del 2021
df_nna = df_unstacked[df_unstacked['edition']==2021]
df_nna.isna().sum()

index
country           0
edition           0
abr               0
co2_prod          2
coef_ineq        39
diff_hdi_phdi    40
eys               2
eys_f             6
eys_m             6
gdi              23
gdi_group        34
gii              25
gni_pc_f         16
gni_pc_m         16
gnipc             2
hdi               4
hdi_f            23
hdi_m            23
ihdi             39
ineq_edu         19
ineq_inc         35
ineq_le           0
le                0
le_f              0
le_m              0
lfpr_f           15
lfpr_m           15
loss             39
mf               38
mmr              11
mys               4
mys_f            16
mys_m            16
phdi             40
pr_f              2
pr_m              2
se_f             18
se_m             18
dtype: int64

In [ ]:
!pip install country_converter
from country_converter import CountryConverter

# Crea un oggetto CountryConverter
cc = CountryConverter()
countries = df_nna['country']
continents = [cc.convert(names=c, to='continent') for c in countries]
df_nna['continent']=continents
df_nna = df_nna[df_nna['continent']!='not found']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 791.5 kB/s eta 0:00:00


In [ ]:
df_nna.columns

Index(['country', 'edition', 'abr', 'co2_prod', 'coef_ineq', 'diff_hdi_phdi',
       'eys', 'eys_f', 'eys_m', 'gdi', 'gdi_group', 'gii', 'gni_pc_f',
       'gni_pc_m', 'gnipc', 'hdi', 'hdi_f', 'hdi_m', 'ihdi', 'ineq_edu',
       'ineq_inc', 'ineq_le', 'le', 'le_f', 'le_m', 'lfpr_f', 'lfpr_m', 'loss',
       'mf', 'mmr', 'mys', 'mys_f', 'mys_m', 'phdi', 'pr_f', 'pr_m', 'se_f',
       'se_m', 'continent'],
      dtype='object', name='index')

In [ ]:
#tolgo le colonne che non ci interessano
df_nna = df_nna.drop(['edition', 'eys', 'eys_f', 'eys_m', 'gdi_group', 'gni_pc_f', 'gni_pc_m', 'gnipc', 'hdi_f', 'hdi_m', 'le', 'le_f', 'le_m', 'mys', 'mys_f', 'mys_m'], axis=1)

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

# Creare una copia del DataFrame
df_encoded = df_nna.copy()

# Creare un codificatore LabelEncoder per le colonne 'country' e 'continent'
label_encoder_country = LabelEncoder()
label_encoder_continent = LabelEncoder()

# Codificare le colonne 'country' e 'continent'
df_encoded['country'] = label_encoder_country.fit_transform(df_encoded['country'])
df_encoded['continent'] = label_encoder_continent.fit_transform(df_encoded['continent'])

# Creare un imputatore iterativo
imputer = IterativeImputer(max_iter=10, random_state=0)

# Fit dell'imputatore sui dati
imputer.fit(df_encoded)

# Trasformazione dei dati con imputazione iterativa
data_imputed = imputer.transform(df_encoded)

# Creare un nuovo DataFrame con i dati imputati
df_imputed = pd.DataFrame(data_imputed, columns=df_encoded.columns)

# Riconversione delle colonne 'country' e 'continent' ai nomi originali
df_imputed['country'] = label_encoder_country.inverse_transform(df_imputed['country'].astype(int))
df_imputed['continent'] = label_encoder_continent.inverse_transform(df_imputed['continent'].astype(int))


In [ ]:
df_imputed.head()

index,country,abr,co2_prod,coef_ineq,diff_hdi_phdi,gdi,gii,hdi,ihdi,ineq_edu,...,lfpr_m,loss,mf,mmr,phdi,pr_f,pr_m,se_f,se_m,continent
0,Afghanistan,82.565,0.312376,19.683581,0.836820,0.68100,0.67800,0.478,0.421510,45.365170,...,66.515000,19.474776,1.380000,638.000000,0.474000,27.215190,72.784810,6.395730,14.865710,Asia
1,Albania,14.517,1.575754,10.767685,7.160804,1.00700,0.14400,0.796,0.710000,12.333440,...,66.154000,10.804020,12.960000,15.000000,0.739000,35.714286,64.285714,95.387505,93.038415,Europe
2,Algeria,11.714,3.534591,19.072875,7.785235,0.88000,0.49900,0.745,0.598000,33.282620,...,64.480000,19.731544,11.210000,112.000000,0.687000,7.481752,92.518248,46.043976,56.927603,Africa
3,Andorra,5.887,6.034945,17.203646,10.924719,0.99486,0.16204,0.858,0.714452,10.008154,...,70.188982,17.954210,14.049766,-21.786029,0.763115,46.428571,53.571429,70.665854,72.422097,Europe
4,Angola,138.402,0.675410,30.412168,1.535836,0.90300,0.53700,0.586,0.407000,34.171440,...,79.071000,30.546075,2.180000,241.000000,0.577000,29.545455,70.454545,28.163900,51.546170,Africa


In [ ]:
df_imputed.isna().sum()

index
country          0
abr              0
co2_prod         0
coef_ineq        0
diff_hdi_phdi    0
gdi              0
gii              0
hdi              0
ihdi             0
ineq_edu         0
ineq_inc         0
ineq_le          0
lfpr_f           0
lfpr_m           0
loss             0
mf               0
mmr              0
phdi             0
pr_f             0
pr_m             0
se_f             0
se_m             0
continent        0
dtype: int64

In [ ]:
df_imputed[df_imputed['continent']=='not found']

index,country,abr,co2_prod,coef_ineq,diff_hdi_phdi,gdi,gii,hdi,ihdi,ineq_edu,...,lfpr_m,loss,mf,mmr,phdi,pr_f,pr_m,se_f,se_m,continent


In [ ]:
df_imputed.min() #i possibili errori sono solo 7, di cui 5 relativi a mmr negativo, li metto a 0

index
country          Afghanistan
abr                    1.584
co2_prod             0.02766
coef_ineq           4.371242
diff_hdi_phdi       0.401606
gdi                    0.496
gii                    0.013
hdi                    0.385
ihdi                    0.24
ineq_edu             0.55011
ineq_inc          -12.479796
ineq_le             1.954472
lfpr_f                 5.995
lfpr_m                43.932
loss                4.357298
mf                 -8.315618
mmr               -90.415414
phdi                   0.358
pr_f                     0.0
pr_m               44.339623
se_f                 6.39573
se_m               13.046886
continent             Africa
dtype: object

In [ ]:
df_imputed['mmr'] = df_imputed['mmr'].apply(lambda x: 2.0 if x < 0 else x)
#2.0 è il min di mmr prima di imputare i dati

In [ ]:
df_imputed.to_csv('/content/drive/MyDrive/Progetto DS Lab/dati_2021.csv', index=False)